# Week 2

## Tuesday May 28, 2019:
* created new SURF_2019 branch of FindMyFriends starting at commit 01753b8fc715284fd8e4fa30621ef5e2c78891e3 (March 3, 2016, Version Bump)
* read through aaa.R and other files to see how pre-CD-Hit FindMyFriends clustered data
* read some methods from [kebab R package](https://www.bioconductor.org/packages/devel/bioc/manuals/kebabs/man/kebabs.pdf) 

I read through the vignette for the package and learned about the main features of FindMyFriends (FMF). FMF reads .fasta (or .faa) files into a pangenome object, and then the pangenome object is manipulated and clustered to determine the pan and core genomes for the input files. 

The first step in FMF is to cluster the genomes based on kmer counts. This is done using a technique called guided pairwise comparison, where a tree is constructed and grouped to represent the relationships between the input sequences. To construct the tree, the genes for each genome are concatenated, kmer count profiles are created for each genome, and then genomes are compared based on their overall kmer count profiles (by calculating the normalized inner product of each vector against each other vector, spectrumkernel from kebab). This similarity matrix is then converted to a distance matrix using the euclidean metric between each genome's kmer count profile, and the distance matrix is clustered (using hclust, method = ward.D2) to generate a dendrogram. Subtrees of the dendrogram are then compared and grouped, and each time a subtree is grouped with another subtree, a random sample is chosen to represent the subtree for the next comparison. This tree can then be further analyzed. 

After reading and learning about FindMyFriends, I installed the package (version 1.1.6) (along with a ton of dependencies!) and tested out the functionality using the examples from the vignette. I used 9 Mycoplasma pneumonia genomes that were provided in the package. 

I was able to group the genes using guided pairwise comparison (gpc) and classify the genes into core, accessory, or singleton categories: ![geneGrouping](images/w2/geneGrouping.png)
I could also get broad statistics about the genomes: ![stats](images/w2/broadStats.png) ![summaryGraphs](images/w2/summaryGraphs.png) ![evo](images/w2/plotEvolution.png)

I was also able to generate two different heatmaps for the genomes. The first groups the genomes based on the percentage of shared genes: ![sharedGenesHeatmap](images/w2/percentSharedGenesHeatmap.png)
I also generated a heatmap of the genomes using cosine similarity distances of the kmer feature vectors for each genome, and it was very similar to the previous heatmaps with the same genome groupings: ![kmerHeatmap](images/w2/5merSimHeatmap.png)

For fun, I also generated a circular phylogenetic tree... ![circularTree](images/w2/circularTree.png)
and a panchromosome map that tries to link the genes based on similar neighboring genes: ![panc](images/w2/panchromosome.png) although I'm not too sure what information (if any) I could actually get from this graph. 

Next, I decided to try to run FindMyFriends for the 26 genomes that were analyzed last week. Code:



In [ ]:
library(FindMyFriends)
library(igraph)

setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files")
genomeFiles <- list.files(getwd(), full.names=TRUE, pattern='*.faa')
pan <- pangenome(genomeFiles[1:27], translated=TRUE, geneLocation='prodigal', lowMem=FALSE)
pan <- gpcGrouping(pan, lowerLimit=0.5)
pan

# pie chart and gene graph
plotStat(pan, type='qual', palette=6)

# see how pangenome composition changes as genomes are added
plotEvolution(pan)

# heatmap based on % of similar genes
plotSimilarity(pan)

# heatmap based on kmer similarity
plotSimilarity(pan, type='kmer', kmerSize=5)

Unfortunately, RStudio crashed while running the gpcGrouping analysis for all 27 genomes. I decided to try to run it with a smaller set of genomes. It took around 19 minutes for the gpcGrouping to finish for the 10 genomes. Results:
![summPlot](images/w2/10sumplot.png)
![evoPlot](images/w2/10evoplot.png)
Clustered based on % gene similarity
![tree](images/w2/phylotree.png)
Split by % genes shared:
![pHM](images/w2/10percentHM.png)
Split by kmer profiles:
![kHM](images/w2/10kmerHM.png)

To check and see if FMF is reproducible, I reran the analysis and got slightly different gene classification ratios. 
![summPlot](images/w2/Re10SummPlot.png)
Clustered based on % gene similarity
![tree](images/w2/ReTree.png)
Split by % genes shared:
![pHM](images/w2/RePHM.png)
Split by kmer profiles:
![kHM](images/w2/ReKmerHM.png)

The heatmaps and tree still look the same, but I will have to continue investigating tomorrow. 
***


## Wednesday May 29, 2019:
* I decided to test the reproducibility of FindMyFriends by running it for 8 genomes 5 times to compare results. 
* while FMF was running, I read about various clustering algorithms to try to get some ideas for a preclustering stept

FindMyFriends version 1.1.6 does not give reproducible results. Each time I ran the program, I got different core, accessory, and singleton counts for the pangenome. ![comparisonGraphs](images/w2/FMF1.1.6SummaryGraphs-1.png)

I think this might arise from Thomas' Guided Pairwise Comparison method: 

"FindMyFriends does away with [computational time and memory restrictions] by introducing a new approach called Guided Pairwise Comparison (GPC). It works by building a tree of the organisms in the set, and recursively combine pangenomes of subtrees into new pangenomes. **Each time two pangenomes are combined, representatives for each gene group in the two pangenomes are chosen at random and compared to each other using kmer similarity.** The resulting grouping is then propagated to genes in the two pangenomes. " 

I was initially told that the random step came from the CD-Hit clustering, but this random representative picking is present in FindMyFriends even before CD-Hit. My next plan of attack is to test an earlier version before GPC was added to see if I can get reproducible results. 

While I was waiting for the graphs to be produced for version 1.1.5, I found the piece of code that randomly samples from the subtree groups to represent each subtree: 

In [ ]:
# In FindMyFriends aaa.R, line 436 in method RecurseCompare()
represent <- sapply(groups, function(x) {
        x[sample.int(length(x), size = 1L)]
    })

The graphs for FMF1.1.5 were also not reproducible:
![1.1.5](images/w2/FMF1.1.5graphs.png)

Luckily, there is a separate way to group genes in FindMyFriends by getting a kmer similarity matrix and then grouping the genes with a separate algorithm. I then tested that method three times and found that it did give reproducible results! Code:

In [ ]:
library(FindMyFriends)
library(igraph)

setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files")
genomeFiles <- list.files(getwd(), full.names=TRUE, pattern='*.faa')
pan <- pangenome(genomeFiles[1:8], translated=TRUE, geneLocation='prodigal', lowMem=FALSE)
panSim <- kmerSimilarity(pan, lowerLimit=0.8, rescale=FALSE)
pan <- graphGrouping(pan, panSim)
pan
plotStat(pan, type='qual', palette=6)

***
## Thursday May 30, 2019:
* try to find a canopy clustering algorithm to test

I found a good python version of canopy clustering:

In [ ]:
# from gdbasset canopy.py gist on github: https://gist.github.com/gdbassett/528d816d035f2deaaca1

from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X, T1, T2, distance_metric='euclidean', filemap=None):
    canopies = dict()
    X1_dist = pairwise_distances(X, metric=distance_metric)
    print(X1_dist)
    canopy_points = set(range(X.shape[0]))
    while canopy_points:
        point = canopy_points.pop()
        i = len(canopies)
        canopies[i] = {"c":point, "points": list(np.where(X1_dist[point] < T2)[0])}
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return canopies


Since FindMyFriends was in R, I decided to go ahead a try to write my own version of the canopy clustering algorithm in R. It was good practice learning to work with data structures in R. My version turned out to be:

In [ ]:
library(FindMyFriends)
library(kebabs)
library(rlist)

setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files")
genomeFiles <- list.files(getwd(), full.names=TRUE, pattern='*.faa')
pan <- pangenome(genomeFiles[1:6], translated=TRUE, geneLocation='prodigal', lowMem=FALSE)
distance_matrix <- kmerSimilarity(pan, kmerSize = 4, lowerLimit=0, rescale=TRUE)

cleaned_names <- c()
for(name in rownames(distance_matrix)){
  print(name)
  print(strsplit(name, " ")[[1]][1])
  cleaned_names <- c(cleaned_names, strsplit(name, " ")[[1]][1])
}
rownames(distance_matrix) <- cleaned_names
colnames(distance_matrix) <- cleaned_names

points <- rownames(distance_matrix)
canopies <- list()
while(length(points) > 0){
  center_point = tail(points, 1)
  print(c("canopy center point: ", center_point))
  points <- points[!(points == center_point)]
  
  T1 <- 0.8
  T2 <- 0.2
  canopy_points <- c()
  for(entry in names(distance_matrix[center_point,])){
    if(distance_matrix[center_point, entry] < T1){
      canopy_points <- c(canopy_points, entry)
    }
    if(distance_matrix[center_point, entry] > 0){
      print(distance_matrix[center_point, entry])
    }
    if(distance_matrix[center_point, entry] < T2){
      points <- points[!(points == entry)]
    }
  }
  print(c("points not in canopy at end", length(points)))
  canopies[[center_point]] <- canopy_points[]
}

Now, I need to test it and make sure that it clusters the data well enough. The matrix seems way too sparse, so I will have to check it out in depth tomorrow.
***